# 📘 Проект: Базовый SQL

## 🎯 Цель проекта

Освоить написание SQL-запросов для анализа данных из реляционных баз.  
На практике вы научитесь:

- использовать `SELECT`, `WHERE`, `ORDER BY`, `GROUP BY`, `HAVING`;
- объединять таблицы с помощью `JOIN`;
- работать с подзапросами;
- применять агрегатные функции (`COUNT`, `SUM`, `AVG`, и др.);


---

## 🗂️ Описание данных

В проекте используется база данных, содержащая информацию о стартапах, инвестициях и венчурных фондах. Всего в базе 7 таблиц:

### 📄 `company`
Сведения о компаниях-стартапах:
- `id` — уникальный идентификатор;
- `name` — название компании;
- `category_code` — категория деятельности (например, `social`, `news`);
- `status` — статус (`operating`, `acquired`, `closed`, `ipo`);
- `founded_at` / `closed_at` — даты основания и закрытия;
- `country_code` — страна регистрации;
- `funding_total` — общая сумма привлечённых инвестиций;
- `investment_rounds` / `funding_rounds` — количество раундов, в которых участвовала компания;
- и другие поля.

### 📄 `fund`
Информация о венчурных фондах:
- `id`, `name`, `founded_at`, `country_code`;
- `investment_rounds` — количество раундов, в которых участвовал фонд;
- `invested_companies` — количество проинвестированных компаний;
- `milestones` — ключевые события в истории фонда.

### 📄 `funding_round`
Данные о раундах инвестиций:
- `id`, `company_id`, `funded_at`;
- `funding_round_type` — тип раунда (`angel`, `venture`, `series_a`, и т.д.);
- `raised_amount` — сумма инвестиций;
- `pre_money_valuation` — оценка стоимости компании до инвестиций;
- `participants`, `is_first_round`, `is_last_round`.

### 📄 `investment`
Связь между фондами и инвестициями:
- `id`, `fund_id`, `company_id`, `funding_round_id`;
- каждый ряд показывает, какой фонд инвестировал в какую компанию в каком раунде.

### 📄 `acquisition`
Информация о поглощениях:
- `id`, `acquiring_company_id`, `acquired_company_id`;
- `price_amount` — сумма сделки;
- `term_code` — форма оплаты (`cash`, `stock`, `cash_and_stock`);
- `acquired_at` — дата сделки.

### 📄 `people`
Сотрудники стартапов:
- `id`, `first_name`, `last_name`, `company_id`.

### 📄 `education`
Образование сотрудников:
- `person_id`, `degree_type`, `instituition`, `graduated_at`.

## 📌 Формат работы

Далее будут представлены задания по SQL.  
Каждое задание включает:

- краткое описание задачи;
- мой SQL-запрос — решение поставленной задачи.

#### 1. Отобразите все записи из таблицы `company` по компаниям, которые закрылись

```sql
SELECT *
FROM company
WHERE status = 'closed';
```
---

#### 2. Отобразите количество привлечённых средств для новостных компаний США. 
Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

```sql
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC;
```
---

#### 3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. 
Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

```sql
SELECT SUM(price_amount) AS total_deals_in_cash
FROM acquisition
WHERE term_code = 'cash' 
  AND acquired_at BETWEEN '2011-01-01' AND '2013-12-31';
```
---
#### 4. Отобразите имя, фамилию и названия аккаунтов людей в поле `network_username`, у которых названия аккаунтов начинаются на 'Silver'.

```sql
SELECT first_name, last_name, network_username
FROM people
WHERE network_username LIKE 'Silver%';
```

---


#### 5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле `network_username` содержат подстроку 'money', а фамилия начинается на 'K'.

```sql
SELECT *
FROM people
WHERE network_username LIKE '%money%' 
  AND last_name LIKE 'K%';
```
---
#### 6. Общая сумма инвестиций по странам 
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране.  
Страну, в которой зарегистрирована компания, можно определить по коду страны. 
Отсортируйте данные по убыванию суммы.

```sql
SELECT c.country_code, SUM(c.funding_total) AS total_investments
FROM company c
GROUP BY c.country_code
ORDER BY total_investments DESC;
```

---

#### 7. Минимальные и максимальные инвестиции по датам  
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.  
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

```sql
SELECT 
    f.funded_at AS funding_date,
    MIN(f.raised_amount) AS min_investment,
    MAX(f.raised_amount) AS max_investment
FROM funding_round f
GROUP BY f.funded_at
HAVING MIN(f.raised_amount) != 0 
   AND MIN(f.raised_amount) != MAX(f.raised_amount)
ORDER BY funding_date;
```

---

#### 8. Категории активности фондов  

Создайте поле с категориями:  
- Фонды, инвестирующие в 100 и более компаний — `high_activity`  
- От 20 до 99 компаний — `middle_activity`  
- Менее 20 компаний — `low_activity`  

Отобразите все поля таблицы `fund` и новое поле с категориями.

```sql
SELECT 
    f.*, 
    CASE 
        WHEN f.invested_companies >= 100 THEN 'high_activity'
        WHEN f.invested_companies >= 20 AND f.invested_companies < 100 THEN 'middle_activity'
        ELSE 'low_activity'
    END AS activity_category
FROM fund f;
```
---

#### 9. Среднее количество инвестиционных раундов по категориям активности  

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого число среднее количество инвестиционных раундов, в которых фонд принимал участие.  
Отсортируйте таблицу по возрастанию среднего.

```sql
SELECT 
    CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 THEN 'middle_activity'
        ELSE 'low_activity'
    END AS activity,
    ROUND(AVG(investment_rounds), 0) AS avg_investment_rounds
FROM fund
GROUP BY activity
ORDER BY avg_investment_rounds;
```

---

#### 10. Топ-10 стран по инвестиционной активности фондов (2010–2012)  

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.  
Исключите страны, в которых минимальное значение равно нулю.  
Выведите топ-10 стран, отсортированных по среднему количеству компаний (по убыванию) и по коду страны (по алфавиту).

```sql
SELECT country_code, 
       MIN(invested_companies), 
       MAX(invested_companies), 
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM founded_at) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, 
         country_code
LIMIT 10;
```

---

#### 11. Сотрудники стартапов и их образование  

Отобразите имя и фамилию всех сотрудников стартапов.  
Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

```sql
SELECT p.first_name, p.last_name, e.instituition
FROM people AS p
LEFT OUTER JOIN education AS e ON p.id = e.person_id;
```

---

#### 12. ТОП-5 компаний по числу учебных заведений у сотрудников  

Для каждой компании найдите количество уникальных учебных заведений, которые окончили её сотрудники.  
Выведите название компании и число университетов.  
Составьте топ-5 компаний по количеству учебных заведений.

```sql
WITH pe AS (
    SELECT p.company_id, e.instituition
    FROM people AS p
    JOIN education AS e ON e.person_id = p.id
)

SELECT c.name, 
       COUNT(DISTINCT pe.instituition)
FROM company AS c
JOIN pe ON c.id = pe.company_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT pe.instituition) DESC
LIMIT 5;
```
---


#### 13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

```sql
SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.status = 'closed'
  AND fr.is_first_round = 1
  AND fr.is_last_round = 1;
```
---

#### 14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

```sql
WITH closed_companies AS (
    SELECT c.id
    FROM company AS c
    JOIN funding_round AS fr ON c.id = fr.company_id
    WHERE c.status = 'closed'
      AND fr.is_first_round = 1
      AND fr.is_last_round = 1
)
SELECT DISTINCT p.id AS employee_id
FROM people AS p
JOIN closed_companies AS cc ON p.company_id = cc.id;
```
---


#### 15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.


```sql
WITH closed_companies AS (
    SELECT c.id
    FROM company AS c
    JOIN funding_round AS fr ON c.id = fr.company_id
    WHERE c.status = 'closed'
      AND fr.is_first_round = 1
      AND fr.is_last_round = 1
)
SELECT DISTINCT p.id AS employee_id, e.instituition
FROM people AS p
JOIN closed_companies AS cc ON p.company_id = cc.id
JOIN education AS e ON p.id = e.person_id;
```
---

#### 16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. 

При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

```sql
WITH 
fr AS (SELECT company_id
       FROM   funding_round
       WHERE  is_first_round = 1
       AND    is_last_round = 1), 

p AS (SELECT DISTINCT p.id
      FROM   people AS p
      WHERE  p.company_id IN (SELECT DISTINCT c.id
                              FROM   company AS c
                              JOIN   fr ON fr.company_id = c.id
                              WHERE  c.status = 'closed')) 

SELECT p.id,
       COUNT(e.instituition) 
FROM   education AS e
JOIN   p ON p.id = e.person_id
GROUP BY p.id;
```
---

####  17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. 

Нужно вывести только одну запись, группировка здесь не понадобится.


```sql
WITH 
fr AS (SELECT company_id
       FROM   funding_round
       WHERE  is_first_round = 1
       AND    is_last_round = 1), 

p AS (SELECT DISTINCT p.id
      FROM   people AS p
      WHERE  p.company_id IN (SELECT DISTINCT c.id
                              FROM   company AS c
                              JOIN   fr ON fr.company_id = c.id
                              WHERE  c.status = 'closed')), 

pe AS (SELECT p.id,
              COUNT(e.instituition)
       FROM   education AS e
       JOIN   p ON p.id = e.person_id
       GROUP BY p.id) 

SELECT AVG(pe.count)
FROM   pe;
```
---

####  18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

```sql
WITH 
fr AS (SELECT company_id
       FROM   funding_round), 

p AS (SELECT DISTINCT p.id
      FROM   people AS p
      WHERE  p.company_id IN (SELECT DISTINCT c.id
                              FROM   company AS c
                              JOIN   fr ON fr.company_id = c.id
                              WHERE  c.name LIKE '%Socialnet%')), 

pe AS (SELECT p.id,
              COUNT(e.instituition)
       FROM   education AS e
       JOIN   p ON p.id = e.person_id
       GROUP BY p.id) 

SELECT AVG(pe.count)
FROM   pe;
```
---

#### 19. Составьте таблицу из полей:

- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

```sql
WITH
fr AS (SELECT *
       FROM   funding_round AS fr
       WHERE  funded_at BETWEEN '2012-01-01' AND '2013-12-31'), 

c AS (SELECT *
      FROM   company
      WHERE  milestones > 6)

SELECT f.name AS name_of_fund, 
       c.name AS name_of_company, 
       fr.raised_amount AS amount 
FROM   investment AS i
JOIN   c ON c.id = i.company_id
JOIN   fund AS f ON f.id = i.fund_id
JOIN   fr ON fr.id = i.funding_round_id;
```
---

#### 20. Выгрузите таблицу, в которой будут такие поля:

- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. 
Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. 
Ограничьте таблицу первыми десятью записями.

```sql
WITH 
c2 AS (SELECT *
       FROM company
       WHERE funding_total > 0
)

SELECT c1.name AS acquiring_company_name, 
       a.price_amount, 
       c2.name AS acquired_company_name, 
       c2.funding_total, 
       ROUND(A.PRICE_AMOUNT / c2.funding_total)
FROM   acquisition AS a
LEFT JOIN company AS c1 ON c1.id = a.acquiring_company_id 
LEFT JOIN company AS c2 ON c2.id = a.acquired_company_id  
WHERE  a.price_amount > 0
AND    c2.funding_total > 0
ORDER BY  a.price_amount DESC, 
          c2.name 
LIMIT     10;
JOIN   fr ON fr.id = i.funding_round_id;
```
---

#### 21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. 

Проверьте, что сумма инвестиций не равна нулю. 
Выведите также номер месяца, в котором проходил раунд финансирования.

```sql
WITH
fr AS (SELECT company_id, 
              EXTRACT(MONTH FROM funded_at) AS funded_month
       FROM   funding_round
       WHERE  funded_at BETWEEN '2010-01-01' AND '2013-12-31'
       AND    raised_amount > 0), 

c AS (SELECT id, 
             name
      FROM   company
      WHERE  category_code = 'social') 

SELECT c.name, 
       fr.funded_month
FROM   c 
JOIN   fr ON fr.company_id = c.id
```
---

#### 22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. 

Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

```sql
WITH

invest AS (SELECT EXTRACT(MONTH FROM fr.funded_at) AS funded_month, 
                  COUNT(DISTINCT f.id) AS count_fund
           FROM   investment AS i 
           JOIN   funding_round AS fr ON fr.id = i.funding_round_id
           JOIN   fund AS f ON f.id = i.fund_id
           WHERE  f.country_code = 'USA'
           AND    fr.funded_at BETWEEN '2010-01-01' AND '2013-12-31'
           GROUP BY funded_month), 

acquired AS (SELECT EXTRACT(MONTH FROM acquired_at) AS acquired_month, 
                    COUNT(acquired_company_id) AS count_company, 
                    SUM(price_amount) AS sum_price_amount 
             FROM   acquisition
             WHERE  acquired_at BETWEEN '2010-01-01' AND '2013-12-31'
             GROUP BY acquired_month) 

SELECT invest.funded_month, 
       invest.count_fund, 
       acquired.count_company,
       acquired.sum_price_amount
FROM   invest
JOIN   acquired ON acquired.acquired_month = invest.funded_month;
```
---

#### 23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. 

Данные за каждый год должны быть в отдельном поле. 
Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

```sql
WITH
inv_2011 AS (SELECT co.country_code, 
                    AVG(co.funding_total) 
             FROM company AS co
             WHERE EXTRACT(YEAR FROM co.founded_at) = 2011
             GROUP BY co.country_code 
             HAVING COUNT(co.id) > 0), 

inv_2012 AS (SELECT co.country_code, 
                    AVG(co.funding_total) 
             FROM company AS co 
             WHERE EXTRACT(YEAR FROM co.founded_at) = 2012 
             GROUP BY co.country_code 
             HAVING COUNT(co.id) > 0),

inv_2013 AS (SELECT co.country_code, 
                    AVG(co.funding_total) 
             FROM company AS co 
             WHERE EXTRACT(YEAR FROM co.founded_at) = 2013 
             GROUP BY co.country_code 
             HAVING COUNT(co.id) > 0)

SELECT inv_2011.country_code,
       inv_2011.avg AS inv_2011,
       inv_2012.avg AS inv_2012,
       inv_2013.avg AS inv_2013
FROM inv_2011
INNER JOIN inv_2012 ON inv_2012.country_code = inv_2011.country_code
INNER JOIN inv_2013 ON inv_2013.country_code = inv_2011.country_code
ORDER BY inv_2011.avg DESC;
```

---

## Заключение

В этом проекте я работала с языком SQL и данными о стартапах, инвестициях и фондах.  
На практике получилось закрепить основные приёмы работы с базами: написание запросов, объединение таблиц, использование агрегатных и оконных функций.

По ходу выполнения заданий стало понятнее, как устроен венчурный рынок, как анализировать активность компаний и фондов, и какие выводы можно делать из данных.

---

## Что получилось

- писать SQL-запросы разной сложности;  
- использовать подзапросы и оконные функции;  
- объединять таблицы с помощью `JOIN`;  
- извлекать срезы данных и проводить первичный анализ.

---

Проект помог систематизировать знания и набраться уверенности в работе с базами данных.
